In [1]:
# import for pyspark sql
from pyspark.sql import SparkSession 
from pyspark.sql import Row, SQLContext
from pyspark.sql.types import *

# imports for streaming
from pyspark import SparkContext 
from pyspark.streaming import StreamingContext
from pyspark import StorageLevel

# mllib imports 
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.classification import LogisticRegressionWithLBFGS
from pyspark.mllib.feature import HashingTF

import time
import pyspark

In [1]:
"""
Spark streamminng Queue of RDDs as a Stream

For testing a Spark Streaming application with test data, one can also create a DStream based on a 
queue of RDDs. Each RDD pushed into the queue will be treated as a batch of data in the DStream,
and processed like a stream.
"""
# Create a local StreamingContext with two working thread and batch interval of 1 second. 
sc = SparkContext("local[2]", "NetworkWordCount").getOrCreate()
ssc = StreamingContext(sc, 1)

# Create the queue through which RDDs can be pushed to a QueueInputDStream 
rddQueue = [] 
for i in range(5): 
    rddQueue += [ssc.sparkContext.parallelize([j for j in range(1, 1001)], 10)] 
 
# Create the QueueInputDStream and use it do some processing 
inputStream = ssc.queueStream(rddQueue) 
mappedStream = inputStream.map(lambda x: (x % 10, 1)) 
reducedStream = mappedStream.reduceByKey(lambda a, b: a + b) 
reducedStream.pprint()  

ssc.start() 
time.sleep(6) 
ssc.stop(stopSparkContext=True, stopGraceFully=True) 

NameError: name 'SparkContext' is not defined

In [13]:
def updateFunction(newValues, runningCount):
    if runningCount is None:
        runningCount = 0
    return sum(newValues, runningCount)  # add the new values with the previous running count to get the new count

# Create a local StreamingContext with two working thread and batch interval of 1 second. 
sc = SparkContext("local[2]", "NetworkWordCount").getOrCreate()
ssc = StreamingContext(sc, 5)
ssc.checkpoint("./checkpoint")

# Create a DStream that will connect to hostname:port, like localhost:9999
lines = ssc.socketTextStream("localhost", 9999)
words = lines.flatMap(lambda line: line.split(" ")) 		# Split each line into words

# Count each word in each batch
pairs = words.map(lambda word: (word, 1))
wordCounts = pairs.reduceByKey(lambda x, y: x + y)

# Print the first ten elements of each RDD generated in this DStream to the console
wordCounts.pprint()
newValues = wordCounts.updateStateByKey(updateFunction)
newValues.pprint()

# Start the computation
ssc.start() 
time.sleep(60)
ssc.stop(stopSparkContext=True, stopGraceFully=True) 

-------------------------------------------
Time: 2021-03-08 07:26:22
-------------------------------------------

-------------------------------------------
Time: 2021-03-08 07:26:22
-------------------------------------------

-------------------------------------------
Time: 2021-03-08 07:26:23
-------------------------------------------

-------------------------------------------
Time: 2021-03-08 07:26:23
-------------------------------------------

-------------------------------------------
Time: 2021-03-08 07:26:24
-------------------------------------------
('interesting', 1)
('talking', 1)
('points', 1)
('Turkish', 1)
('President', 1)
('Erdogan’s', 1)
('spox', 1)
('', 1)
('new', 1)
('chapter', 1)
...

-------------------------------------------
Time: 2021-03-08 07:26:24
-------------------------------------------
('interesting', 1)
('talking', 1)
('points', 1)
('Turkish', 1)
('President', 1)
('Erdogan’s', 1)
('spox', 1)
('', 1)
('new', 1)
('chapter', 1)
...

----------------